# Memory

https://python.langchain.com/api_reference/langchain/memory.html

챗봇으로 하여금 대화(상태)를 '기억'하게끔 한다

Memory maintains Chain state, incorporating context from past runs.


In [ ]:
# Langchain 의 memory 계층
#  BaseMemory --> BaseChatMemory --> <name>Memory  # Examples: ZepMemory, MotorheadMemory

In [1]:
# OpenAI 사에서 제공하는 '기본 API' 도 랭체인 없이 사용 가능.
# 메모리 지원하지 않는다.  이전 대화 기억 못함.  stateless 하다!

# ChatGPT 서비스 는 '메모리' 기능이 탑재되어 있다.
# 챗봇이 이전의 대화 내용과 질문을 기억하고 답할수 있다.

# 기본 import

In [16]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.prompts.chat import ChatPromptTemplate

# ConversationBufferMemory

In [2]:
# ConversationBufferMemory
# 대화 내용 '전체'를 저장하는 메모리

# 장점: 단순하다

# 단점:
# => 매번 요청할때마다 '이전 대화 기록 전체' 를 같이 보내야 함.
#  그래야 모델이 전에 일어났던 대화를 보고 이해 할수 있다.
#  대화내용이 길어질수록 메모리도 계속 커지니까 성능적으로도 & 비용적으로도 비효율적이다.

In [3]:
# v0.3
from langchain.memory.buffer import ConversationBufferMemory
# https://python.langchain.com/api_reference/langchain/memory/langchain.memory.buffer.ConversationBufferMemory.html#conversationbuffermemory



In [4]:
memory = ConversationBufferMemory()

memory.save_context(
    {"input": "Hi!"},
    {'output': 'How are you?'}
)

memory.load_memory_variables({})  # history buffer 리턴

C:\Users\User\AppData\Local\Temp\ipykernel_23164\346152885.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()


{'history': 'Human: Hi!\nAI: How are you?'}

## return_messages=True
history 에  AIMessage 와 HumanMessage 로 저장된다.

In [5]:
memory = ConversationBufferMemory(return_messages=True)

memory.save_context(
    {"input": "Hi!"},
    {'output': 'How are you?'}
)

memory.load_memory_variables({}) 

{'history': [HumanMessage(content='Hi!', additional_kwargs={}, response_metadata={}),
  AIMessage(content='How are you?', additional_kwargs={}, response_metadata={})]}

In [7]:
memory.save_context(
    {"input": "Hi!"},
    {'output': 'How are you?'}
)

memory.load_memory_variables({}) 

{'history': [HumanMessage(content='Hi!', additional_kwargs={}, response_metadata={}),
  AIMessage(content='How are you?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Hi!', additional_kwargs={}, response_metadata={}),
  AIMessage(content='How are you?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Hi!', additional_kwargs={}, response_metadata={}),
  AIMessage(content='How are you?', additional_kwargs={}, response_metadata={})]}

# ConversationBufferWindowMemory

In [8]:
# v0.3
from langchain.memory.buffer_window import ConversationBufferWindowMemory
# https://python.langchain.com/api_reference/langchain/memory/langchain.memory.buffer_window.ConversationBufferWindowMemory.html#conversationbufferwindowmemory


In [9]:
# ConversationBufferWindowMemory 는 대화의 '특정 부분만' 을 저장하는 메모리.

# 장점:
#   메모리를 특정 크기로 유지할 수 있다!
#   따라서 모든 대화 내용을 저장하지 않아도 된다!

# 단점:
#   챗봇이 전체 대화가 아닌 '최근 대화' 에만 집중하게 된다.

In [10]:
memory = ConversationBufferWindowMemory(
    return_messages=True,
    k=4,   # 버퍼 윈도우 사이즈,  몇개의 메세지를 저장할지 지정.
)

C:\Users\User\AppData\Local\Temp\ipykernel_23164\2681781434.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(


In [11]:
# 도우미 함수 하나 준비.  여러 메세지를 추가하기 편리하니까
def add_message(input, output):
  memory.save_context({"input": input}, {"output": output})

In [12]:
add_message("1", "1")
add_message("2", "2")
add_message("3", "3")
add_message("4", "4")

In [13]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='1', additional_kwargs={}, response_metadata={}),
  AIMessage(content='1', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='2', additional_kwargs={}, response_metadata={}),
  AIMessage(content='2', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='3', additional_kwargs={}, response_metadata={}),
  AIMessage(content='3', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='4', additional_kwargs={}, response_metadata={}),
  AIMessage(content='4', additional_kwargs={}, response_metadata={})]}

In [14]:
add_message("5", "5")
memory.load_memory_variables({})

{'history': [HumanMessage(content='2', additional_kwargs={}, response_metadata={}),
  AIMessage(content='2', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='3', additional_kwargs={}, response_metadata={}),
  AIMessage(content='3', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='4', additional_kwargs={}, response_metadata={}),
  AIMessage(content='4', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='5', additional_kwargs={}, response_metadata={}),
  AIMessage(content='5', additional_kwargs={}, response_metadata={})]}

# ConversationSummaryMemory

In [ ]:
# ConversationSummaryMemory 는 LLM 을 사용하여 대화의 요약본 (summary) 생성

In [17]:
llm = ChatOpenAI(temperature=0.1)

In [18]:
# v0.3
from langchain.memory.summary import ConversationSummaryMemory
# https://python.langchain.com/api_reference/langchain/memory/langchain.memory.summary.ConversationSummaryMemory.html#langchain.memory.summary.ConversationSummaryMemory

# Continually summarizes the conversation history.
# The summary is updated after each conversation turn.
# The implementations returns a summary of the conversation history
# which can be used to provide context to the model.



In [19]:
memory = ConversationSummaryMemory(llm = llm)

C:\Users\User\AppData\Local\Temp\ipykernel_23164\3817587095.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(llm = llm)


In [20]:
# 함수 하나 만들어 두고..

def get_history():
  return memory.load_memory_variables({})

In [21]:
# message 추가
add_message(
    "Hi I'm John, I live in South Korea",    # input
    "Wow that is so cool!"  # output : AI 답변
    )

In [22]:
# 또 message 추가
add_message(
    "South Korea is so pretty",
    "I wish I could go!!!")

In [23]:
get_history()

{'history': 'John introduces himself as living in South Korea. The AI responds by expressing admiration for his location, wishing it could go there.'}

In [ ]:
# ↑ 대화를 '요약' 한 내용으로 기억하고 있다
# 대화의 turn 이 길어질수로 summary 가 각 메세지를 효율적으료 '요약(압축)' 해준다

# ConversationSummaryBufferMemory

In [24]:
# v0.3
from langchain.memory.summary_buffer import ConversationSummaryBufferMemory
# https://python.langchain.com/api_reference/langchain/memory/langchain.memory.summary_buffer.ConversationSummaryBufferMemory.html#langchain.memory.summary_buffer.ConversationSummaryBufferMemory

# Buffer with summarizer for storing conversation memory.
# Provides a running summary of the conversation together with
#  the most recent messages in the conversation under the constraint
#   that the total number of tokens in the conversation does not exceed a certain limit.


In [ ]:
# ConversationSummaryBufferMemory 는
#   ConversationBufferMemory 와 ConversationSummaryMemory 의 결합형

# 메모리에 보내온 '메세지의 수'를 지정하여 저장한다.
# 오래된 메세지들 또한 '요약' 하여 저장함.

In [25]:
llm = ChatOpenAI(temperature=0.1)

In [26]:
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=150,   # 최대 가용한 메세지 토큰수 (메세지 요약되기 전)
    return_messages=True,
)

C:\Users\User\AppData\Local\Temp\ipykernel_23164\3924822954.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryBufferMemory(


In [27]:
add_message(
    "Hi I'm John, I live in South Korea",    # input
    "Wow that is so cool!"  # output : AI 답변
    )
get_history()

{'history': [HumanMessage(content="Hi I'm John, I live in South Korea", additional_kwargs={}, response_metadata={}),
  AIMessage(content='Wow that is so cool!', additional_kwargs={}, response_metadata={})]}

In [28]:
add_message(
    "South Korea is so pretty",
    "I wish I could go!!!")
get_history()

{'history': [HumanMessage(content="Hi I'm John, I live in South Korea", additional_kwargs={}, response_metadata={}),
  AIMessage(content='Wow that is so cool!', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='South Korea is so pretty', additional_kwargs={}, response_metadata={}),
  AIMessage(content='I wish I could go!!!', additional_kwargs={}, response_metadata={})]}

In [29]:
add_message(
    "How far is Korea from Argentina?",
    "I don't know! Super far!"
)
get_history()

{'history': [HumanMessage(content="Hi I'm John, I live in South Korea", additional_kwargs={}, response_metadata={}),
  AIMessage(content='Wow that is so cool!', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='South Korea is so pretty', additional_kwargs={}, response_metadata={}),
  AIMessage(content='I wish I could go!!!', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='How far is Korea from Argentina?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="I don't know! Super far!", additional_kwargs={}, response_metadata={})]}

In [33]:
# 아래 셀을 여러차레 해보자  약 (3,4번?)
# =>실제 '요약'이 발생할때면 Model IO 가 발생하기 때문에 시간이 좀 걸리는게 느껴질거다!

add_message(
    "How far is Brazil from Argentina?",
    "I don't know! Super far!"
)
get_history()

{'history': [SystemMessage(content='The human introduces himself as John and mentions he lives in South Korea.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Wow that is so cool!', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='South Korea is so pretty', additional_kwargs={}, response_metadata={}),
  AIMessage(content='I wish I could go!!!', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='How far is Korea from Argentina?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="I don't know! Super far!", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='How far is Brazil from Argentina?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="I don't know! Super far!", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='How far is Brazil from Argentina?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="I don't know! Super far!", additional_kwargs={}

In [ ]:
# limit 에 도달하면, 오래된 메세지들이 요약되고 있을 것을 확인할수 있다. (SystemMessage 확인)

# ★그러나 '요약' 이라는 과정은 API 를 사용한다는 사실을 명심하세요.
# ★'요약' 동작은 비용 지출이 발생되는 부분입니다.

# ConversationKGMemory
Knowledge Graph Memory

In [34]:
# v0.3
from langchain_community.memory.kg import ConversationKGMemory
# https://python.langchain.com/api_reference/community/memory/langchain_community.memory.kg.ConversationKGMemory.html

# Knowledge graph conversation memory.
# Integrates with external knowledge graph to store and retrieve information about knowledge triples in the conversation.

In [ ]:
# 대화중에 '엔티티'의 knowledge graph 를 형성한다 => 가장 중요한 것들만 추출한 요약본.
# knowledge graph 는 history 를 가지고 오지 않는다.  대신 '엔티티' 를 가지고 옴


In [35]:
memory = ConversationKGMemory(
    llm=llm,
    return_messages=True,
)

In [36]:
add_message(
    "Hi I'm John, I live in South Korea",    # input
    "Wow that is so cool!"  # output : AI 답변
    )

In [37]:
memory.load_memory_variables({'input': 'who is John'})

{'history': [SystemMessage(content='On John: John lives in South Korea.', additional_kwargs={}, response_metadata={})]}

In [38]:
# 여기에 메세지를 더해보자
add_message("John likes kimchi", "Wow that is so cool!")

In [39]:
memory.load_memory_variables({'input': 'What does John like'})

{'history': [SystemMessage(content='On John: John lives in South Korea. John likes kimchi.', additional_kwargs={}, response_metadata={})]}